# This notebook runs a Monte Carlo propagation in ADAM

In [ ]:
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
from adam import AdamProcessingService
from adam import BatchPropagationResults
import time
import os

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [ ]:
# ConfigManager loads the config set up via adamctl.
# See the README at https://github.com/B612-Asteroid-Institute/adam_home/blob/master/README.md
config = ConfigManager().get_config('mcdemo')
auth_rest = AuthenticatingRestProxy(RestRequests(config['url']), config['token'])
aps = AdamProcessingService(auth_rest)

## Example Inputs

The PropagationParameters sent to the propagation API will operate on either Keplerian or Cartesian elements.

In [ ]:
# Keplerian elements state vector
keplerian_elements = {
    'semi_major_axis_km': 448793612,
    'eccentricity': 0.1,
    'inclination_deg': 90,
    'ra_of_asc_node_deg': 91,
    'arg_of_pericenter_deg': 92,
    'true_anomaly_deg': 93,
    'gm': 132712440041.9394
}

# Uncertainties on elements
keplerian_sigma = {
    'semi_major_axis': 100,
    'eccentricity': 0.001,
    'inclination': 1,
    'ra_of_asc_node': 2,
    'arg_of_pericenter': 3,
    'true_anomaly': 4,
}

# Cartesian state vector
state_vec = [-150874809.2,
             -187234595.3,
             -73785026.7,
             14.64403935,
             -11.75744819,
             -5.583528281]

cartesian_sigma = {
    "x": 10,
    "y": 10,
    "z": 10,
    "x_dot": 1,
    "y_dot": 1,
    "z_dot": 1    
}


### Set Parameters

Commented parameters are optional. Uncomment to use.

In [ ]:
propagation_params = PropagationParams({
    'start_time': '2001-01-02T01:13:46.62Z',  # propagation start time in ISO format
    'end_time': '2010-12-31T01:13:46.62Z',    # propagation end time in ISO format

    #'keplerianSigma': keplerian_sigma,
    'description': 'Jupyter Example',
    'cartesianSigma' : cartesian_sigma,       # comment out if using keplerian
    'monteCarloDraws': 10,
    'propagationType': 'MONTE_CARLO',
    'stopOnImpact': True,
    'step_size': 86400,
    'stopOnCloseApproach': False,
    'stopOnImpactDistanceMeters': 500000,
    'closeApproachRadiusFromTargetMeters': 7000000000
})

opm_params = OpmParams({
    'epoch': '2001-01-02T01:13:46Z',
    # 'keplerian_elements': keplerian_elements,
    'state_vector': state_vec,                # comment out if using keplerian
})


### Submit and Run Propagation

In [ ]:
batch_run = aps.execute_batch_propagation(config['workspace'], propagation_params, opm_params)
# batch_run = BatchPropagationResults.fromRESTwithRawIds(auth_rest, config['workspace'], 'efaa4ecc-c6d0-4d57-a769-cd400a3a0997')
print(batch_run)

### Get Status

In [ ]:
print(batch_run.check_status())
batch_run.wait_for_complete(print_waiting = True)
print()
print(batch_run.check_status())

### Get Number of Runs

In [ ]:
runs_count = batch_run.get_result_ephemeris_count()
print(f'# Runs: {runs_count}')

### Get Summary Statistics

In [ ]:
stats = batch_run.get_summary()
print(stats)

### Get Ephemeris for specific run

In [ ]:
eph = batch_run.get_result_ephemeris(2)
print(eph)

### Get ending state vector

In [ ]:
end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.CLOSE_APPROACH)
print("Close State Vectors")
print(f'First close end state: {end_state_vectors[0]}')

end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.IMPACT)
print("Impact State Vectors")
print(len(end_state_vectors))
print(f'First impact end state: {end_state_vectors[0]}')

end_state_vectors = batch_run.get_final_positions(BatchPropagationResults.PositionOrbitType.MISS)
print("Miss State Vectors")
print(len(end_state_vectors))
print(f'First miss end state: {end_state_vectors[0]}')
